In [1]:
import numpy as np
import os,sys
import pickle

import Tools

from DataGenerator import genTrainValidFolds

In [ ]:
class KNN():
    def __init__(self):
        pass
    
    def train(self,x, y):
        self.trainX = x
        self.trainY = y
#         print (self.trainY[30870])
#         print (self.trainY.shape, self.trainY.dtype)
        
    def similarity(self, validX, L1=None, L2=None, Cosine=None):
        if L1:
            distance = np.sum(np.abs(self.trainX - validX), axis = 1)
            min_index = np.argmin(distance)
            return min_index
        if L2:
            distance = np.sqrt(np.sum(np.square(self.trainX - validX), axis=1))
            min_index = np.argmin(distance)
            return min_index
    
    def predict(self, validX):
        num_examples = validX.shape[0]
        predY = np.zeros(shape=(num_examples, self.trainY.shape[1]), dtype=self.trainY.dtype)
        for vrow in range(num_examples):
            min_index = self.similarity(validX[vrow,:], L1='L1')
#             print (min_index)
            predY[vrow] = self.trainY[int(min_index)]
            if (vrow%100==0 and vrow!=0):
                print ('Running for vrow: ', vrow)
        print (predY.shape)

    def main(self):
        for trainData, trainLabels, validData, validLabels in genTrainValidFolds():
            print ('Validation Data and Labels shape: ', validData.shape, validLabels.shape)
            print ('Training Data and Labels shape: ', trainData.shape, trainLabels.shape)
            self.train(trainData, trainLabels)
            self.predict(validData)
            break
        
KNN().main()

Running i is : 0
Running j is :  1
Running j is :  2
Running j is :  3
Running j is :  4
Running j is :  5
Running j is :  6
Running j is :  7
Running j is :  8
Running j is :  9
Validation Data and Labels shape:  (5000, 1024) (5000, 10)
Training Data and Labels shape:  (45000, 1024) (45000, 10)


In [11]:
a = np.array([[1,2,3,4],[2,3,4,5]]) - [1,1,1,1]
print (a)
print (np.abs(a))
print (np.sum(np.abs(a), axis=1))

[[0 1 2 3]
 [1 2 3 4]]
[[0 1 2 3]
 [1 2 3 4]]
[ 6 10]
